In [49]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, LinearRegression




In [50]:
data = pd.read_csv('cfs_2012_pumf_csv.txt')
print(data.shape)
data.head()

(4547661, 20)


,SHIPMT_ID,ORIG_STATE,ORIG_MA,ORIG_CFS_AREA,DEST_STATE,DEST_MA,DEST_CFS_AREA,NAICS,QUARTER,SCTG,MODE,SHIPMT_VALUE,SHIPMT_WGHT,SHIPMT_DIST_GC,SHIPMT_DIST_ROUTED,TEMP_CNTL_YN,EXPORT_YN,EXPORT_CNTRY,HAZMAT,WGT_FACTOR
0,1,25,148,25-148,25,148,25-148,333,2,35,14,2178,11,14,17,N,N,N,N,208.5
1,2,42,428,42-428,6,41740,06-41740,311,3,35,14,344,11,2344,2734,N,N,N,N,193.3
2,3,26,220,26-220,47,314,47-314,322,2,27,4,4197,5134,470,579,N,N,N,N,51.2
3,4,20,556,20-556,20,556,20-556,323,1,29,4,116,6,3,3,N,N,N,N,238.7
4,5,12,99999,12-99999,12,99999,12-99999,4235,3,33,5,388,527,124,201,N,N,N,N,398.1


## Featuring Engineering

In [42]:
data.corr()

,SHIPMT_ID,ORIG_STATE,ORIG_MA,DEST_STATE,DEST_MA,NAICS,QUARTER,MODE,SHIPMT_VALUE,SHIPMT_WGHT,SHIPMT_DIST_GC,SHIPMT_DIST_ROUTED,WGT_FACTOR
SHIPMT_ID,1.000000,0.000169,0.000288,-0.000182,0.000448,-0.000120,0.000054,-0.001140,0.000535,-0.000163,-0.000462,-0.000498,-0.000499
ORIG_STATE,0.000169,1.000000,0.054526,0.507648,0.028042,0.001577,0.000456,-0.017253,0.002470,0.013203,-0.033262,-0.029491,-0.005572
ORIG_MA,0.000288,0.054526,1.000000,0.026407,0.378136,-0.010465,0.006329,-0.073226,-0.002220,0.020197,-0.040448,-0.034713,-0.024695
DEST_STATE,-0.000182,0.507648,0.026407,1.000000,0.041635,-0.003382,0.001995,-0.024948,0.001516,0.005646,-0.056825,-0.051738,-0.002635
DEST_MA,0.000448,0.028042,0.378136,0.041635,1.000000,-0.003606,0.004708,-0.052183,-0.003937,0.002627,-0.060194,-0.061423,-0.009709
NAICS,-0.000120,0.001577,-0.010465,-0.003382,-0.003606,1.000000,-0.001581,0.005819,0.000586,0.000041,-0.012451,-0.012856,0.010893
QUARTER,0.000054,0.000456,0.006329,0.001995,0.004708,-0.001581,1.000000,-0.007362,0.000545,0.000543,-0.007935,-0.008187,-0.005600
MODE,-0.001140,-0.017253,-0.073226,-0.024948,-0.052183,0.005819,-0.007362,1.000000,-0.000079,0.009697,0.335222,0.347859,0.079089
SHIPMT_VALUE,0.000535,0.002470,-0.002220,0.001516,-0.003937,0.000586,0.000545,-0.000079,1.000000,0.105667,0.000968,0.001794,-0.001867
SHIPMT_WGHT,-0.000163,0.013203,0.020197,0.005646,0.002627,0.000041,0.000543,0.009697,0.105667,1.000000,-0.001638,0.002145,-0.004383


In [51]:
# transform NAICS values into 8 major categories: mining (except oil & gas), manufacturing, 
# wholesale, storage, fuel, online shopping, publishing, administration
def NAICS(x):
    if x >= 300 and x <= 400:
        return 'Manufacturing'
    elif x >= 4200 and x <= 4300:
        return 'Wholesale'
    elif x == 212:
        return 'Mining'
    elif x == 4541:
        return 'Online shopping'
    elif x == 45431:
        return 'Fuel'
    elif x == 4931:
        return 'Storage'
    elif x == 5111:
        return 'Publishing'
    else:
        return 'Administration'
    
data['NAICS_type'] = data['NAICS'].map(lambda x: NAICS(x))
data['NAICS_type'].value_counts()

Manufacturing      2149701
Wholesale          1882220
Mining              140933
Storage             101644
Fuel                101200
Online shopping      86257
Publishing           53137
Administration       32569
Name: NAICS_type, dtype: int64

In [52]:
# transform MODE values into 4 major categories:
def MODE(x):
    if x >= 3 and x <= 5:
        return 'Truck'
    elif x == 6:
        return 'Rail'
    elif x >= 7 and x <= 10 or x == 101:
        return 'Water'
    elif x == 11:
        return 'Air'
    elif x == 12:
        return 'Pipeline'
    elif x == 14:
        return 'Parcel'
    elif x == 15:
        return 'Truck and rail'
    else:
        return 'Other/multiple'

data['transportation_type'] = data['MODE'].map(lambda x: MODE(x))
data['transportation_type'].value_counts()
# data['MODE'].value_counts()

Truck             3231969
Parcel            1165297
Air                 68809
Rail                38458
Truck and rail      19070
Other/multiple      16489
Water                3896
Pipeline             3673
Name: transportation_type, dtype: int64

In [55]:
data.groupby('transportation_type').agg({'SHIPMT_VALUE': 'mean'})

,SHIPMT_VALUE
transportation_type,
Air,1.734364e+05
Other/multiple,3.082203e+05
Parcel,1.772388e+03
Pipeline,1.701407e+06
Rail,1.252864e+05
Truck,1.459159e+04
Truck and rail,8.561352e+04
Water,1.074966e+06


In [48]:
X = pd.get_dummies(data['NAICS_type'])

# min_max_scaler = preprocessing.MinMaxScaler()
y = data['SHIPMT_VALUE']

model = LinearRegression().fit(X, y)
model.score(X, y)

9.615652989303225e-05

In [83]:
X = pd.concat([pd.get_dummies(data['NAICS_type']), pd.get_dummies(data['transportation_type']), 
               data['SHIPMT_WGHT'], data['SHIPMT_DIST_ROUTED']], ignore_index=True, axis=1)

# min_max_scaler = preprocessing.MinMaxScaler()
# y = min_max_scaler.fit_transform(data['SHIPMT_VALUE'].values.reshape(-1,1))
y = data['SHIPMT_VALUE']

model = LinearRegression()
model.fit(X, y)
print(model.score(X, y))
model.coef_

0.01314645404552195


array([ 1.51995970e+04, -7.16886974e+03,  1.73799344e+04, -5.89562771e+04,
        5.96930859e+03,  3.62774735e+02,  2.67330416e+04,  4.80490356e+02,
       -9.90416176e+04, -8.87938764e+04, -2.68576175e+05,  1.02483134e+06,
       -3.44027277e+05, -2.55514926e+05, -1.97553047e+05,  2.28675582e+05,
        1.14478360e-01, -2.37587053e+00])

In [56]:
from statsmodels.formula.api import ols
import statsmodels.api as sm
# why can we use ols in this case?
anova = ols('SHIPMT_VALUE~transportation_type',data=data).fit()
anova.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           SHIPMT_VALUE   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2274.
Date:                Mon, 25 Nov 2019   Prob (F-statistic):               0.00
Time:                        20:30:10   Log-Likelihood:            -6.9645e+07
No. Observations:             4547661   AIC:                         1.393e+08
Df Residuals:                 4547653   BIC:                         1.393e+08
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                              1.734e+05   4130.013     41.994      0.000    1.65e+05    1.82e+05
transportation_type[T.Other/multiple]  1.348e+05   9393.419     14.349      0.000    1.16e+05    1.53e+05
transportation_type[T.Parcel]         -1.717e+05   4250.200    -40.390      0.000    -1.8e+05   -1.63e+05
transportation_type[T.Pipeline]        1.528e+06   1.83e+04     83.283      0.000    1.49e+06    1.56e+06
transportation_type[T.Rail]           -4.815e+04   6897.491     -6.981      0.000   -6.17e+04   -3.46e+04
transportation_type[T.Truck]          -1.588e+05   4173.746    -38.058      0.000   -1.67e+05   -1.51e+05
transportation_type[T.Truck and rail] -8.782e+04   8865.814     -9.906      0.000   -1.05e+05   -7.04e+04
transportation_type[T.Water]           9.015e+05   1.78e+04     50.531      0.000    8.67e+05    9.36e+05
================================================================================
Omnibus:                 27333656.233   Durbin-Watson:                     2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   7688714460228492.000
Skew:                         426.312   Prob(JB):                           0.00
Kurtosis:                  201437.825   Cond. No.                           47.5
================================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
# def quantile_feature(feature, num_bins):
#     mean_over_feature = data.groupby(feature).agg({'SHIPMT_VALUE': 'mean'})
#     dest_state_quartiles = pd.DataFrame(pd.qcut(mean_over_feature['SHIPMT_VALUE'], num_bins))
#     dest_state_quartiles.rename(columns={'SHIPMT_VALUE': feature + '_QT'}, inplace=True)
#     data.merge(pd.DataFrame(dest_state_quartiles), left_on=feature, right_index=True)

# data = quantile_feature('ORIG_STATE', 4)
# data = quantile_feature('DEST_STATE', 4)
# data.head()

In [45]:
# pd.get_dummies(data.DEST_STATE)

## Examining Regression Coefficients

In [59]:
y = data[['SHIPMT_VALUE']]
X = data[['SHIPMT_WGHT', 'SHIPMT_DIST_ROUTED']]

# 'NAICS', 'QUARTER', 'MODE',

In [94]:
X

,SHIPMT_WGHT,SHIPMT_DIST_ROUTED
0,11,17
1,11,2734
2,5134,579
3,6,3
4,527,201
...,...,...
4547656,133,152
4547657,29887,683
4547658,137,16
4547659,1240,22


In [63]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

linreg = LinearRegression()
linreg.fit(X_train, y_train)
print(linreg.coef_)
print(linreg.intercept_)
print(linreg.score(X_train, y_train))

[[0.11876464 2.70002516]]
[12990.82599514]
0.009563235505794077
